In [1]:
from model import unet_model
import tensorflow as tf
import glob
from functools import partial
from utils import dice_coef, dice_bce_loss

In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [4]:
#Зададим размер batch и размер изображения
BATCH_SIZE = 16
IMG_SHAPE  = (256, 256)
OUTPUT_SIZE = (960, 1280)

CLASSES = 6

#Цвета для отображения сегментации
Colors = ['black',
          'blue',
          'green',
          'sky',
          'cyan',
          'orange',
          'pink',
          'red']
rgb_colors = [(0,   0,   0),
              (0, 0,   255),
              (0, 255,  0),
              (0, 255,  0),
              (0,   255, 0),
              (0,   0,   255),
              (255, 165, 0),
              (255, 192, 203),
              (0,   255, 255),
              (255, 0,   255)]

In [12]:
images = sorted(glob.glob('./dataset/images/*.jpg'))
masks = sorted(glob.glob('./dataset/masks/*.bmp'))

In [35]:
#Функция для загрузки изображения
def load_img(image, mask):
    #Загружаем и препроцессим изображения
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image)
    image = tf.image.resize(image, IMG_SHAPE)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = image / 255.0

    #Загружаем и препроцессим маски изображений
    mask = tf.io.read_file(mask)
    mask = tf.io.decode_bmp(mask)
    # mask = tf.image.grayscale_to_rgb(mask) #Если маски сохранены в GRAY
    mask = tf.image.rgb_to_grayscale(mask)
    mask = tf.image.resize(mask, IMG_SHAPE)
    mask = tf.image.convert_image_dtype(mask, tf.float32)

    #Сохраним маски для каждого класса
    masks = []
    for i in range(CLASSES):
        masks.append(tf.where(tf.equal(mask, float(i)), 1.0, 0.0))
    masks = tf.stack(masks, axis=2)
    masks = tf.reshape(masks, IMG_SHAPE + (CLASSES,))
    return image, masks

#Аугментация изобажений
def aug_img(image, masks):
    random_crop = tf.random.uniform((), 0.3, 1)
    image = tf.image.central_crop(image, random_crop)
    masks = tf.image.central_crop(masks, random_crop)
    random_flip = tf.random.uniform((), 0, 1)
    if random_flip >= 0.5:
        image = tf.image.flip_left_right(image)
        masks = tf.image.flip_left_right(masks)
    image = tf.image.resize(image, IMG_SHAPE)
    masks = tf.image.resize(masks, IMG_SHAPE)
    return image, masks

In [36]:
#Формируем датасет
dataset = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(images),
                               tf.data.Dataset.from_tensor_slices(masks)))

#Загружаем дату
dataset = dataset.map(load_img, num_parallel_calls=tf.data.AUTOTUNE)

#Аугментация (увеличим датасет в 50 раз)
# dataset = dataset.repeat(50)
# dataset = dataset.map(aug_img, num_parallel_calls=tf.data.AUTOTUNE)

In [37]:
#Размер train выборки
train_size = 1200

#Делим на train и test
train_dataset = dataset.take(train_size).cache()
test_dataset = dataset.skip(train_size).take(len(dataset) - train_size).cache()

train_dataset = train_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [30]:
#Функция для подсчёта DICE коэффициента
def dice_coef(y_pred, y_true):
    y_pred = tf.unstack(y_pred, axis=3)
    y_true = tf.unstack(y_true, axis=3)
    dice_summ = 0

    for i, (a_y_pred, b_y_true) in enumerate(zip(y_pred, y_true)):
        dice_calculate = (2 * tf.math.reduce_sum(a_y_pred * b_y_true) + 1) /\
         (tf.math.reduce_sum(a_y_pred + b_y_true) + 1)

        dice_summ += dice_calculate
    avg_dice = dice_summ/CLASSES
    return avg_dice


#Функция для подсчета DICE loss
def dice_loss(y_pred, y_true):
    d_loss = 1 - dice_coef(y_pred, y_true)
    return d_loss


#binary_crossentropy - дает хорошую сходимость модели при сбалансированном наборе данных
#DICE - хорошо в задачах сегментации но плохая сходимость
#Плохо сбалансированные данные, но хорошие реузьтаты:
# Binary crossentropy + 0.25 * DICE

def dice_bce_loss(y_pred, y_true):
    total_loss = 0.25 * dice_loss(y_pred, y_true) + tf.keras.losses.binary_crossentropy(y_pred, y_true)
    return total_loss

In [39]:
#Иницилизируем модель
model = unet_model(IMG_SHAPE, CLASSES)

# tf.keras.utils.plot_model(model, show_shapes=True)
#Компилируем модель
model.compile(optimizer='adam',
                  loss=[dice_bce_loss],
                  metrics=[dice_coef])

In [40]:
EPOCHS = 10

#Обучение
model.fit(train_dataset,
          validation_data=test_dataset,
          epochs=EPOCHS,
          initial_epoch=0)

Epoch 1/10
11/75 [===>..........................] - ETA: 5:09 - loss: 0.9115 - dice_coef: 0.0964Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Karpo\PycharmProjects\cv_group6\venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Karpo\AppData\Local\Temp\ipykernel_11764\2367473858.py", line 4, in <module>
    model.fit(train_dataset,
  File "C:\Users\Karpo\PycharmProjects\cv_group6\venv\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "C:\Users\Karpo\PycharmProjects\cv_group6\venv\Lib\site-packages\tensorflow\python\eager\context.py", line 1415, in get_graph_debug_info
    pywrap_tfe.TFE_ContextGetGraphDebugInfo(self._handle, name, buffer_)
MemoryError: bad allocation

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Karpo\PycharmProjects\cv_group6\venv\Lib\site-packages\IPython\core\interactiveshell.py", line 2142, in sh

In [ ]:
model.save('./checkpoints/checkpoint_best_unet_like_model_10epochs')